In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Define symbolic variables
var('a b c')
assume(a > 0, b > 0, c > 0)

def get_reps(k):
    """
    Returns the su(2) representation matrices E1, E2, E3 for weight k.
    Dimension of the matrix is k+1. 
    test
    """
    dim = k + 1
    # Initialize matrices in Symbolic Ring (SR)
    E1 = Matrix(SR, dim, dim, 0)
    E2 = Matrix(SR, dim, dim, 0)
    E3 = Matrix(SR, dim, dim, 0)
    
    for l in range(dim):
        # E1: Diagonal elements (weights)
        E1[l, l] = I * (k - 2*l)
        
        # E2: Real part of ladder operators
        if l > 0:
            E2[l-1, l] = -l           # Contribution from P_{l-1}
        if l < k:
            E2[l+1, l] = k - l        # Contribution from P_{l+1}
        
        # E3: Imaginary part of ladder operators
        if l > 0:
            E3[l-1, l] = -l * I
        if l < k:
            E3[l+1, l] = -(k - l) * I
            
    return E1, E2, E3

def get_curv(a, b, c):
    """
    Calculates the eigenvalues r_ij of the curvature operator R on 2-vectors.
    Note: Based on the definition R(X^Y) = 0.5 * sum(E_i ^ R(X,Y)E_i),
    these eigenvalues are NEGATIVE for the standard sphere (a=b=c=1).
    """
    r12 = 3*a**2*b**2/c**2 - a**2*c**2/b**2 - b**2*c**2/a**2 - 2*a**2 - 2*b**2 + 2*c**2
    r13 = -a**2*b**2/c**2 + 3*a**2*c**2/b**2 - b**2*c**2/a**2 - 2*a**2 + 2*b**2 - 2*c**2
    r23 = -a**2*b**2/c**2 - a**2*c**2/b**2 + 3*b**2*c**2/a**2 + 2*a**2 - 2*b**2 - 2*c**2
    return r12, r13, r23

def Delta(k, a_val=None, b_val=None, c_val=None, numeric=False):
    # DEBUG PRINT
    if numeric:
        print(f"DEBUG: Delta recalled with k={k}, a={a_val}, b={b_val}, c={c_val}")
    """
    Constructs the Hodge-Laplacian matrix Delta for representation k.
    If a_val, b_val, c_val are provided, substitutes them.
    If numeric=True, converts to RDF (Real Double Field) for speed.
    """
    # 1. Get representation matrices
    E1, E2, E3 = get_reps(k)
    dim_rep = E1.nrows()
    
    # Identity matrices
    id_rep = identity_matrix(SR, dim_rep)
    id_form = identity_matrix(SR, 3) # 1-forms are 3D
    
    # 2. Connection matrices N (Levi-Civita on 1-forms)
    # Note: N_i represents -Gamma_{i, .}^.
    # Terms like (ab/c + ac/b - bc/a) appear frequently.
    
    N1 = -1 * Matrix(SR, [
        [0, 0, 0],
        [0, 0, -(-a*b/c - a*c/b + b*c/a)],
        [0, -a*b/c - a*c/b + b*c/a, 0]
    ])
    
    N2 = -1 * Matrix(SR, [
        [0, 0, -(a*b/c - a*c/b + b*c/a)],
        [0, 0, 0],
        [a*b/c - a*c/b + b*c/a, 0, 0]
    ])
    
    N3 = -1 * Matrix(SR, [
        [0, -(a*b/c - a*c/b - b*c/a), 0],
        [a*b/c - a*c/b - b*c/a, 0, 0],
        [0, 0, 0]
    ])
    
    # 3. Casimir Operators
    # Casimir on V_k (E_i are anti-hermitian, so squares are negative)
    Cas_rep = (a**2 * E1*E1 + b**2 * E2*E2 + c**2 * E3*E3)
    
    # Casimir on 1-forms
    Cas_form = (N1*N1 + N2*N2 + N3*N3).apply_map(lambda x: x.expand())
    
    # Mixed interaction term
    Mixed = 2*a*E1.tensor_product(N1) + 2*b*E2.tensor_product(N2) + 2*c*E3.tensor_product(N3)
    
    # 4. Curvature term q(R) (Ricci)
    r12, r13, r23 = get_curv(a, b, c)
    
    # q(R) acts diagonally. Since r_ij are negative, (-r_ij - r_ik) yields positive Ricci curvature.
    qR = diagonal_matrix([-r12-r13, -r23-r12, -r23-r13])
    
    # 5. Assemble Delta = nabla*nabla + 2q(R)
    # Rough Laplacian = -(Cas_rep + Cas_form + Mixed)
    RoughLaplacian = -(Cas_rep.tensor_product(id_form) + id_rep.tensor_product(Cas_form) + Mixed)
    CurvatureTerm = id_rep.tensor_product(qR)
    
    Delta = RoughLaplacian + CurvatureTerm
    
    # 6. Substitution / Numeric conversion
    if a_val is not None and b_val is not None and c_val is not None:
        Delta = Delta.subs(a=a_val, b=b_val, c=c_val)
        if numeric:
            Delta = Delta.change_ring(CDF) 
            
    return Delta

[4*b^2*c^2/a^2             0             0]
[            0 4*a^2*c^2/b^2             0]
[            0             0 4*a^2*b^2/c^2]


In [2]:
from IPython.display import display, Math

# Settings
max_k = 5

print(f"--- HODGE-LAPLACIAN MATRICES (k=0 to {max_k}) ---")

for k in range(max_k + 1):
    # Get symbolic matrix
    M = Delta(k, numeric=False)
    
    # Dimensions
    rows = M.nrows()
    cols = M.ncols()
    
    # Title
    display(Math(f"\\textbf{{Representation }} k = {k} \\quad (\\text{{Dimension }} {rows} \\times {cols})"))
    
    # Generate LaTeX
    # We use a scaling wrapper to make it smaller
    latex_matrix = latex(M)
    
    # For larger matrices, we wrap it in \tiny or \scriptsize
    if k > 0:
        latex_code = f"\\tiny {latex_matrix}"
    else:
        latex_code = f"{latex_matrix}"
        
    # Display
    try:
        display(Math(latex_code))
    except Exception as e:
        print(f"Could not render matrix for k={k}: {e}")
        
    print("-" * 60)

--- HODGE-LAPLACIAN MATRICES (k=0 to 5) ---


<IPython.core.display.Math object>

<IPython.core.display.Math object>

------------------------------------------------------------


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

------------------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

------------------------------------------------------------


In [4]:
from IPython.display import display, Math

# Ensure variables are defined
var('a b c')
assume(a > 0, b > 0, c > 0)

def show_symbolic_eigenvalues(k):
    print(f"\n{'='*60}")
    print(f"SYMBOLIC EIGENVALUES FOR k = {k}")
    print(f"{'='*60}")
    
    # 1. Get the symbolic matrix
    M = Delta(k, numeric=False)
    
    # 2. Compute eigenvalues
    # This returns a list of eigenvalues (repeated by algebraic multiplicity)
    print("Computing characteristic polynomial and roots... (this may take a moment)")
    evals = M.eigenvalues()
    
    # 3. Process and Display
    # We group them to show multiplicity cleanly, but since symbolic comparison 
    # is hard, we just list them and try to simplify.
    
    for i, eig in enumerate(evals):
        print(f"\nEigenvalue {i+1}:")
        
        # Try to simplify the expression
        try:
            simple_eig = eig.simplify_full()
        except:
            simple_eig = eig # Fallback if simplification fails
            
        # Display as LaTeX
        display(Math(latex(simple_eig)))

# --- EXECUTE ---


show_symbolic_eigenvalues(0)

show_symbolic_eigenvalues(1)
show_symbolic_eigenvalues(2)



SYMBOLIC EIGENVALUES FOR k = 0
Computing characteristic polynomial and roots... (this may take a moment)

Eigenvalue 1:


<IPython.core.display.Math object>


Eigenvalue 2:


<IPython.core.display.Math object>


Eigenvalue 3:


...

<IPython.core.display.Math object>


Eigenvalue 8:


<IPython.core.display.Math object>


Eigenvalue 9:


<IPython.core.display.Math object>

In [11]:
import numpy as np

def hunt_the_culprit(a_val, b_val, c_val, label="Metric"):
    print(f"\n{'='*60}")
    print(f"FAHNDUNG NACH DEM ÜBELTÄTER: {label} (a={a_val}, b={b_val}, c={c_val})")
    print(f"{'='*60}")

    # 1. Baseline bestimmen (k=0 und k=1)
    l0 = get_min_eigenvalue(0, a_val, b_val, c_val)
    l1 = get_min_eigenvalue(1, a_val, b_val, c_val)
    baseline = min(l0, l1)
    
    print(f"Min(k=0): {l0:.4f}")
    print(f"Min(k=1): {l1:.4f}")
    print(f"--> Hürde (Baseline): {baseline:.4f}")

    # 2. k=2 komplett analysieren
    # Wir holen uns ALLE Eigenwerte, nicht nur das Minimum
    M_k2 = Delta(2, a_val, b_val, c_val, numeric=True)
    evals_k2 = sorted([e.real() for e in M_k2.eigenvalues()])
    
    min_k2 = evals_k2[0]

    # 3. Vergleich
    if min_k2 < baseline:
        diff = baseline - min_k2
        print(f"\n🚨 ALARM! Übeltäter bei k=2 gefunden!")
        print(f"Wert des Übeltäters:  {min_k2:.4f}")
        print(f"Er ist kleiner um:    {diff:.4f}")
        
        # Zeige die Top 3 kleinsten von k=2 an, falls es mehrere sind
        print(f"\nDie kleinsten Eigenwerte bei k=2:")
        print(np.round(evals_k2[:5], 4))
    else:
        print(f"\n✅ Entwarnung. Kein Dip bei k=2.")
        print(f"Min(k=2) = {min_k2:.4f} >= {baseline:.4f}")

# --- TESTLÄUFE ---

# 1. Der "Generic" Fall aus deinem Plot (grüne Linie)
hunt_the_culprit(2, 3, 5, "Generic (2,3,5)")

# 2. Der "Extreme" Fall (rote Linie)
hunt_the_culprit(1, 10, 1, "Extreme (b=10)")

# 3. Zum Vergleich: Runde Sphäre (sollte sauber sein)
hunt_the_culprit(1, 1, 1, "Round Sphere")


FAHNDUNG NACH DEM ÜBELTÄTER: Generic (2,3,5) (a=2, b=3, c=5)
Min(k=0): 0.0400
Min(k=1): 102.0000
--> Hürde (Baseline): 0.0400

✅ Entwarnung. Kein Dip bei k=2.
Min(k=2) = 7.4540 >= 0.0400

FAHNDUNG NACH DEM ÜBELTÄTER: Extreme (b=10) (a=1, b=10, c=1)
Min(k=0): 0.0400
Min(k=1): 102.0000
--> Hürde (Baseline): 0.0400

✅ Entwarnung. Kein Dip bei k=2.
Min(k=2) = 7.4540 >= 0.0400

FAHNDUNG NACH DEM ÜBELTÄTER: Round Sphere (a=1, b=1, c=1)
Min(k=0): 0.0400
Min(k=1): 102.0000
--> Hürde (Baseline): 0.0400

✅ Entwarnung. Kein Dip bei k=2.
Min(k=2) = 7.4540 >= 0.0400
